# MovieLens を用いたレコメンデーションの実装
このノートでは、レコメンデーションの実装例を示します。

In [1]:
import os
from urllib.request import urlopen

#  MovieLensのデータを取得.
if not os.path.exists("ml-100k.zip"):
    url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
    with urlopen(url) as res:
        with open("data/ml-100k.zip", "wb") as f:
            f.write(res.read())

In [2]:
# Zipファイルを解凍.
from shutil import unpack_archive
unpack_archive("data/ml-100k.zip", "data/", "zip")

In [72]:
import numpy as np
import pandas as pd

# 評価データを読み込み（ u1.base は学習用データ）
udata = pd.read_csv("data/ml-100k/u1.base", delimiter="\t", names=("user", "movie", "rating", "timestamp"))
udata.tail()

,user,movie,rating,timestamp
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275
79999,943,1330,3,888692465


In [73]:
# 行が映画、列がユーザーのマトリックスを作成.
data = np.zeros((udata["movie"].max(), udata["user"].max()), dtype=np.int)
data.shape

(1682, 943)

In [74]:
# 上記で作成したマトリックスに、データを流し込む.
for i, row in udata.iterrows():
    # ratingが3以上のみを対象にしよう（好評価のみ）
    if row["rating"] >= 3:
        data[row["movie"]-1][row["user"]-1] = 1

In [75]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,1,1,0,0,0,1,0,0,0,0,...,0,1,1,0,1,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
# 評価データ数
# 評価データのうち82%は、3以上をつけているよう。みんな良い評価をつけたがる。
df.astype(bool).sum(axis=1).sum()

66103

In [77]:
from scipy.spatial.distance import cosine

item1 = np.array([0,1,0])
item2 = np.array([1,0,1])
sim = 1- cosine(item1, item2)
print(sim)

# 個別のアイテム同士で、距離を求めてみる（ここではコサイン距離）
item1 = data[0]
item2 = data[1]
sim = 1 - cosine(item1, item2)
print(sim)

0.0
0.325207858278


In [78]:
# 上記の雰囲気で、総当たりで全アイテムの距離を計算する.

from scipy.spatial.distance import pdist
d = pdist(data, "cosine")
# 類似度 = 1 - コサイン距離
d = 1 - d

# 結果を行列に変換します（上記だとベクトルで見辛い！！）
from scipy.spatial.distance import squareform
d = squareform(d)
# nan ができるので、0に補正します.
d[np.isnan(d)] = 0

# 表示してみる.
print(d)

[[ 0.          0.32520786  0.27172635 ...,  0.          0.05322463
   0.05322463]
 [ 0.32520786  0.          0.20689728 ...,  0.          0.10910895
   0.10910895]
 [ 0.27172635  0.20689728  0.         ...,  0.          0.          0.14586499]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.05322463  0.10910895  0.         ...,  0.          0.          0.        ]
 [ 0.05322463  0.10910895  0.14586499 ...,  0.          0.          0.        ]]


In [79]:
# 試しに推薦をして見ます.

# ここでちょっとしたトリックで、自分自身は「-1」に補正して、類似度を最低にします.
d = d - np.eye(d.shape[0])
print(d)

[[-1.          0.32520786  0.27172635 ...,  0.          0.05322463
   0.05322463]
 [ 0.32520786 -1.          0.20689728 ...,  0.          0.10910895
   0.10910895]
 [ 0.27172635  0.20689728 -1.         ...,  0.          0.          0.14586499]
 ..., 
 [ 0.          0.          0.         ..., -1.          0.          0.        ]
 [ 0.05322463  0.10910895  0.         ...,  0.         -1.          0.        ]
 [ 0.05322463  0.10910895  0.14586499 ...,  0.          0.         -1.        ]]


In [80]:

# class Container(object):
#     def __init__(self, id, sim):
#         self.id = id
#         self.sim = sim
#     def __repr__(self):
#         return "{0} : {1}".format(self.id, self.sim)

# 例えば、映画1（movie=1）に類似する映画を、類似度の高い順に並べます.
recommends = {}
for index, sim in enumerate(d[0]):
    if sim > 0:
        recommends[index + 1] = sim # indexは0始まりになっているので、+1してidに変換する.
        
recommends = sorted(list(recommends.items()), key=lambda r:r[1], reverse=True)
from pprint import pprint
pprint(recommends[:10])


[(50, 0.62828380959743046),
 (181, 0.60179361066116654),
 (121, 0.56361171748207761),
 (117, 0.55909527338435139),
 (222, 0.5448072284259402),
 (405, 0.53934291268451406),
 (257, 0.52993874949054487),
 (237, 0.52990539045000284),
 (7, 0.52766307860331085),
 (151, 0.51909505604442208)]


In [179]:
# 指定したユーザーへレコメンドするアイテムを10個出力する関数
def get_recommend_items(user_id):
    # 指定ユーザーが評価した映画一覧を取得.
    used = set(df[user_id].nonzero()[0].tolist())
    # レコメンドを作成.
    candidates = {}
    for movie_id in used:
        for index, sim in enumerate(d[movie_id]):
            if sim > 0:
                candidates[index] = sim
    candidates = sorted(list(candidates.items()), key=lambda r:r[1], reverse=True)
    # すでに閲覧済は除く.
    recommends = []
    for c in candidates:
        if c[0] not in used:
            recommends.append(c)
#         else:
#             print("USED:", c)
#     print(recommends)
    
    return [r[0] for r in recommends[:10]]

# 試しにUser_ID=100の人
recommends = get_recommend_items(100)
print(recommends)

[236, 110, 297, 49, 120, 281, 470, 814, 14, 404]


In [180]:
# テストデータを読み込む.（ u1.test は学習用データ）
utest = pd.read_csv("data/ml-100k/u1.test", delimiter="\t", names=("user", "movie", "rating", "timestamp"))
utest.tail()

,user,movie,rating,timestamp
19995,458,648,4,886395899
19996,458,1101,4,886397931
19997,459,934,3,879563639
19998,460,10,3,882912371
19999,462,682,5,886365231


In [181]:
# 行が映画、列がユーザーのマトリックスを作成.
test = np.zeros((utest["movie"].max(), utest["user"].max()), dtype=np.int)
test.shape

(1591, 462)

In [182]:
# 上記で作成したマトリックスに、データを流し込む.
for i, row in utest.iterrows():
    # ratingが3以上のみを対象にしよう（好評価のみ）
    if row["rating"] >= 3:
        test[row["movie"]-1][row["user"]-1] = 1

In [183]:
df_test = pd.DataFrame(test)
df_test.shape

(1591, 462)

In [184]:
# 試しに、userId=1の人でテスト.
used = set(df_test[0].nonzero()[0].tolist())
recommends = set(get_recommend_items(0))
used & recommends

{257, 271}

In [185]:
# 無事にレコメンドができたようだ！

In [186]:
# 続けて他の人もやってみよう.
all = 0
good = 0
for user_id in range(df_test.shape[1]):
    used = set(df_test[user_id].nonzero()[0].tolist())
    recommends = set(get_recommend_items(user_id))
    items = used & recommends
    good += (1 if items else 0)
    all += 1

print("全件={0}, 成功数={1}, 成功率={2}%".format(all, good, good * 100 // all))

全件=462, 成功数=334, 成功率=72%
